In [ ]:

import pandas as pd
from src.executor.match_recognize import match_recognize
import pandas as pd

# Define the data
data = [
    ('cust_1', '2020-05-11', 100),
    ('cust_1', '2020-05-12', 200),
    ('cust_2', '2020-05-13',   8),
    ('cust_1', '2020-05-14', 100),
    ('cust_2', '2020-05-15',   4),
    ('cust_1', '2020-05-16',  50),
    ('cust_1', '2020-05-17', 100),
    ('cust_2', '2020-05-18',   6),
]

# Create DataFrame
df = pd.DataFrame(data, columns=['customer_id', 'order_date', 'price'])

# Convert order_date column to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Display the DataFrame
print(df)

query_basic = """
SELECT
  customer_id,
  run_start,
  run_end
FROM memory.default.orders
MATCH_RECOGNIZE (
  PARTITION BY customer_id
  ORDER BY order_date
  MEASURES
    FIRST(A.order_date) AS run_start,
    LAST(B.order_date)  AS run_end
  PATTERN (A B+)
  DEFINE
    B AS B.price < PREV(price)
) AS t;





"""

print("Test 1:")
output_df = match_recognize(query_basic, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT customer_id, run_start, run_end FROM memory.default.orders MATCH_RECOGNIZE ( PARTITION BY customer_id ORDER BY order_date MEASURES FIRST(A.order_date) AS run_start, LAST(B.order_date) AS run_end PATTERN (A B+) DEFINE B AS B.price > PREV(price) ) AS t;
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=customer_id, metadata={}), SelectItem(expression=run_start, metadata={}), SelectItem(expression=run_end, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['customer_id'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='order_date', ordering='ASC', nulls_ordering=None)])
DEBUG:s

  customer_id order_date  price
0      cust_1 2020-05-11    100
1      cust_1 2020-05-12    200
2      cust_2 2020-05-13      8
3      cust_1 2020-05-14    100
4      cust_2 2020-05-15      4
5      cust_1 2020-05-16     50
6      cust_1 2020-05-17    100
7      cust_2 2020-05-18      6
Test 1:
Pattern value: 'A B+'
Pattern variables {'A'} have no DEFINE conditions - defaulting to TRUE (always match)
Pattern value: 'A B+'
Pattern variables {'A'} have no DEFINE conditions - defaulting to TRUE (always match)
DEBUG: About to create NFA with start=0, accept=1, states_count=10
DEBUG: State 0 before NFA creation: transitions=0, epsilon=[2]
DEBUG: State 1 before NFA creation: transitions=0, epsilon=[]
DEBUG: State 2 before NFA creation: transitions=0, epsilon=[3]
DEBUG: State 3 before NFA creation: transitions=1, epsilon=[]
DEBUG: State 4 before NFA creation: transitions=0, epsilon=[7]
DEBUG: State 5 before NFA creation: transitions=1, epsilon=[]
DEBUG: State 6 before NFA creation: transition